# Пайплайн <a id='top'></a>

## Imports

In [1]:
import yaml

import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from data_transformer import DataTransformer
from models import BoostingModel
from scorer import MoneyScorer

2022-04-16 03:06:58.365908: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-16 03:06:58.365971: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# PATHS

In [2]:
DATA_PATH = 'data/all_england_v2.2.csv'
FEATURES_PATH = 'data/features.yaml'

## Load data

In [3]:
raw_train_data = pd.read_csv(DATA_PATH)

with open(FEATURES_PATH) as f:
    all_features_dict = yaml.safe_load(f)

## Preprocess data

In [4]:
def base_data_preprocess(data):
    
    preprocessed_data = data.copy()
    
    preprocessed_data = preprocessed_data.fillna(1.01)
    preprocessed_data['day'] = pd.to_datetime(preprocessed_data.date, format='%d.%m.%Y')
    preprocessed_data.date = preprocessed_data.day.values.astype(np.int64) // 10 ** 9
    preprocessed_data = preprocessed_data.sort_values(by='date')
    preprocessed_data = preprocessed_data.drop(columns=['link', 'country', 'time', 'month'])
    preprocessed_data.loc[preprocessed_data.league == 'premier-league', 'league'] = 'premier-league-2021-2022'
    preprocessed_data.loc[preprocessed_data.league == 'championship', 'league'] = 'championship-2021-2022'
    preprocessed_data.loc[:, 'season'] = preprocessed_data.apply(set_season, axis=1)
    preprocessed_data.loc[:, 'league'] = preprocessed_data.apply(set_league, axis=1)
    
    # ТЕКУЩАЯ ВЕРСИЯ
    preprocessed_data.loc[preprocessed_data.league == 'premier-league', 'league_level'] = 1


    return preprocessed_data

In [5]:
def set_league(row):
    l = row.league
    l = l.split('-')[:-2:1]
    return "-".join(l)
    
def set_season(row):
    s = row.league
    s = s.split('-')[-2:]
    return "-".join(s)    

In [6]:
train_data = base_data_preprocess(raw_train_data)

train_data.tail()

,date,day_of_week,season,league,league_level,home_team,away_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,home_handicap_0,away_handicap_0,home_handicap_minus_15,away_handicap_minus_15,home_handicap_minus_1,away_handicap_minus_1,home_handicap_plus_15,away_handicap_plus_15,home_handicap_plus_1,away_handicap_plus_1,home_first_half_handicap_0,away_first_half_handicap_0,home_first_half_handicap_minus_15,away_first_half_handicap_minus_15,home_first_half_handicap_minus_1,away_first_half_handicap_minus_1,home_first_half_handicap_plus_15,away_first_half_handicap_plus_15,home_first_half_handicap_plus_1,away_first_half_handicap_plus_1,home_second_half_handicap_0,away_second_half_handicap_0,home_second_half_handicap_minus_15,away_second_half_handicap_minus_15,home_second_half_handicap_minus_1,away_second_half_handicap_minus_1,home_second_half_handicap_plus_15,away_second_half_handicap_plus_15,home_second_half_handicap_plus_1,away_second_half_handicap_plus_1,home_first_half_win_rate,draw_first_half_rate,away_first_half_win_rate,home_second_half_win_rate,draw_second_half_rate,away_second_half_win_rate,home_double_chance_first_half_rate,away_double_chance_first_half_rate,no_draw_first_half_rate,home_double_chance_second_half_rate,away_double_chance_second_half_rate,no_draw_second_half_rate,total_first_half_over_05_rate,total_first_half_under_05_rate,total_first_half_over_1_rate,total_first_half_under_1_rate,total_first_half_over_15_rate,total_first_half_under_15_rate,total_first_half_over_2_rate,total_first_half_under_2_rate,total_second_half_over_05_rate,total_second_half_under_05_rate,total_second_half_over_1_rate,total_second_half_under_1_rate,total_second_half_over_15_rate,total_second_half_under_15_rate,total_second_half_over_2_rate,total_second_half_under_2_rate,both_team_to_score_first_half_yes,both_team_to_score_first_half_no,both_team_to_score_second_half_yes,both_team_to_score_second_half_no,odd,even,odd_first,odd_second,even_first,even_second,correct_score10,correct_score20,correct_score21,correct_score30,correct_score31,correct_score32,correct_score40,correct_score41,correct_score00,correct_score11,correct_score22,correct_score33,correct_score01,correct_score02,correct_score12,correct_score03,correct_score13,correct_score23,correct_score04,correct_score14,home_home,home_draw,home_away,draw_home,draw_draw,draw_away,away_home,away_draw,away_away,home_scored,away_scored,home_first_half_scored,away_first_half_scored,home_second_half_scored,away_second_half_scored,home_manager_working_days,away_manager_working_days,home_manager_name,away_manager_name,home_manager_start_date,home_manager_birthday,home_manager_country,away_manager_start_date,away_manager_birthday,away_manager_country,home_squad_size,home_average_age,home_amount_of_foreigners,home_e_market_value,home_total_market_value,away_squad_size,away_average_age,away_amount_of_foreigners,away_e_market_value,away_total_market_value,home_stadium,home_stadium_capacity,home_city,away_city,is_derby,day
1691,1649548800,Sunday,2021-2022,premier-league,1,Manchester City,Liverpool FC,1.98,3.84,3.78,1.30,1.86,1.31,1.05,10.59,1.23,4.08,1.35,3.17,1.79,2.11,2.29,1.65,2.87,1.43,1.62,2.30,1.50,2.64,3.57,1.28,2.88,1.44,1.09,7.19,1.11,6.26,1.64,2.26,7.57,1.07,5.92,1.12,1.02,12.18,1.03,10.78,1.66,2.22,6.20,1.12,4.69,1.19,1.04,11.30,1.06,9.29,2.71,2.29,3.94,2.45,2.71,3.58,1.22,1.44,1.60,1.26,1.52,1.42,1.34,3.24,1.67,2.16,2.62,1.47,4.85,1.17,1.17,4.33,1.32,3.06,1.84,1.85,2.81,1.37,4.02,1.24,3.00,1.39,1.89,1.87,2.08,1.98,1.75,1.83,7.32,8.56,7.36,13.12,11.62,17.37,35.0,30.0,10.67,6.04,10.60,31.70,10.27,15.24,9.91,31.70,20.76,23.82,101.0,100.0,3.00,13.0,29.0,5.00,6.00,8.5,21.0,13

In [7]:
train_data.shape

(4482, 156)

# Разбиение фичей на группы [ПРОПУСТИТЬ](#numeric)

In [ ]:
enumerated_features = dict()

for idx, feature in enumerate(list(train_data.columns)):
    enumerated_features[idx] = feature

In [ ]:
enumerated_features

In [ ]:
base_features = list(train_data.iloc[0:2,0:7].columns)
base_features.append(enumerated_features[124])
base_features.append(enumerated_features[125])
base_features.append(enumerated_features[156])

base_features

In [ ]:
squad_features = list(train_data.iloc[0:2,141:151].columns)
squad_features

In [ ]:
city_features = list(train_data.iloc[0:2,151:156].columns)
city_features

In [ ]:
result_coef_features = [enumerated_features[7], enumerated_features[8], enumerated_features[9]]
result_coef_features

In [ ]:
double_chance_features = list(train_data.filter(like='chance', axis=1).columns)
double_chance_features

In [ ]:
total_coef_features = list(train_data.filter(like='total_over', axis=1).columns) + list(train_data.filter(like='total_under', axis=1).columns)
total_coef_features

In [ ]:
handicap_features = list(train_data.iloc[0:2,27:37].columns)
handicap_features

In [ ]:
half_features = list(train_data.iloc[0:2,37:88].columns)
half_features[:10]

In [ ]:
odd_features = list(train_data.iloc[0:2,89:95].columns)

odd_features

In [ ]:
correct_score_features = list(train_data.filter(like='correct_score', axis=1).columns)
correct_score_features

In [ ]:
time_match_features = list(train_data.iloc[0:2,115:124].columns)
time_match_features

In [ ]:
both_scored_features = list(train_data.iloc[0:2,85:89].columns)
both_scored_features += list(train_data.iloc[0:2,126:130].columns)
both_scored_features.append(enumerated_features[25])
both_scored_features.append(enumerated_features[26])

both_scored_features

In [ ]:
manager_features = tuple(train_data.filter(like='manager', axis=1).columns)

manager_features

In [ ]:
print(prepare_for_yaml(manager_features))

<a id='numeric'></a>
## Запускать отсюда)

In [8]:
def prepare_for_yaml(features_list) -> str:
    
    result_yaml = ''
    
    for feature in features_list:
        result_yaml += f"- {feature}\n"
        
    return result_yaml

In [9]:
numeric_features = tuple(train_data.select_dtypes(include=['int', 'float']).columns)
print(prepare_for_yaml(numeric_features))

- date
- league_level
- home_win_rate
- draw_rate
- away_win_rate
- home_double_chance_rate
- away_double_chance_rate
- no_draw_rate
- total_over_1_rate
- total_under_1_rate
- total_over_15_rate
- total_under_15_rate
- total_over_2_rate
- total_under_2_rate
- total_over_25_rate
- total_under_25_rate
- total_over_3_rate
- total_under_3_rate
- total_over_35_rate
- total_under_35_rate
- both_team_to_score_yes
- both_team_to_score_no
- home_handicap_0
- away_handicap_0
- home_handicap_minus_15
- away_handicap_minus_15
- home_handicap_minus_1
- away_handicap_minus_1
- home_handicap_plus_15
- away_handicap_plus_15
- home_handicap_plus_1
- away_handicap_plus_1
- home_first_half_handicap_0
- away_first_half_handicap_0
- home_first_half_handicap_minus_15
- away_first_half_handicap_minus_15
- home_first_half_handicap_minus_1
- away_first_half_handicap_minus_1
- home_first_half_handicap_plus_15
- away_first_half_handicap_plus_15
- home_first_half_handicap_plus_1
- away_first_half_handicap_plus_1


In [10]:
categorical_features = tuple(train_data.select_dtypes(include=['object']).columns)
print(prepare_for_yaml(categorical_features))

- day_of_week
- season
- league
- home_team
- away_team
- home_manager_name
- away_manager_name
- home_manager_country
- away_manager_country
- home_stadium
- home_city
- away_city



In [11]:
money_features = tuple(['away_e_market_value', 'away_total_market_value', 'home_e_market_value', 'home_total_market_value'])
print(prepare_for_yaml(money_features))

- away_e_market_value
- away_total_market_value
- home_e_market_value
- home_total_market_value



<a id='transformer'></a>
# Трансформер 

[наверх](#top)

In [ ]:
transformer_context = {'data':train_data,
                       'cat_features':categorical_features,
                       'num_features':numeric_features,
                       'money_features':money_features,
                       'grouped_features':all_features_dict
                      }

transformer = DataTransformer(transformer_context)

In [ ]:
train, val, test, decode_teams, teams_labels = transformer.run_logic() 

In [ ]:
test.head()

In [ ]:
train.target.value_counts()

In [ ]:
print(f"Class_0 rate {train.target.value_counts()[0] / train.shape[0]}")
print(f"Class_1 rate {train.target.value_counts()[1] / train.shape[0]}")
print(f"Class_3 rate {train.target.value_counts()[3] / train.shape[0]}")

In [ ]:
corr_df = train.corr()
corr_df.style.background_gradient(cmap='Reds')

In [ ]:
corr_df[['target']].style.background_gradient(cmap='Reds')

# Проверить данные на:

- мультиколлинеарность
- forward/backward selection
- 


# Переход к [модели](#model)

## Generating some features and testing hypotheses

# Оставим только "не-коэффициенты" и декодируем названия команд 

In [12]:
def _set_target(row):
    """ Set target feature from score """

    if row.home_scored > row.away_scored:
        return 3
    elif row.home_scored == row.away_scored:
        return 1
    else:
        return 0

train_data_base = train_data[all_features_dict['base_features'] + all_features_dict['squad_features']].copy()
train_data_base['target'] = train_data_base.apply(_set_target, axis=1)
train_data_base.tail()

,date,day_of_week,season,league,league_level,home_team,away_team,home_scored,away_scored,day,home_squad_size,home_average_age,home_amount_of_foreigners,home_e_market_value,home_total_market_value,away_squad_size,away_average_age,away_amount_of_foreigners,away_e_market_value,away_total_market_value,target
1691,1649548800,Sunday,2021-2022,premier-league,1,Manchester City,Liverpool FC,2,2,2022-04-10,22,27.5,15,4.360000e+09,9.593000e+10,27,27.1,19,3.335000e+09,9.005000e+10,1
1702,1649548800,Sunday,2021-2022,premier-league,1,Brentford FC,West Ham United,2,0,2022-04-10,25,25.8,22,9.240000e+08,2.309000e+10,24,28.8,16,1.453000e+09,3.487500e+10,3
1515,1649548800,Sunday,2021-2022,premier-league,1,Leicester City,Crystal Palace,2,1,2022-04-10,27,27.0,18,1.907000e+09,5.148000e+10,26,27.5,14,1.000000e+09,2.599500e+10,3
4369,1649548800,Sunday,2021-2022,championship,2,Fulham FC,Coventry City,1,3,2022-04-10,26,26.9,20,5.650000e+08,1.470000e+10,25,25.4,10,7.960000e+05,1.990000e+09,0
4444,1649635200,Monday,2021-2022,championship,2,Huddersfield Town,Luton Town,2,0,2022-04-11,28,26.0,10,1.340000e+08,3.765000e+09,26,27.1,11,8.080000e+05,2.100000e+09,3


In [13]:
for feature in money_features:
    train_data_base[f"log_{feature}"] = train_data_base[feature].apply(np.log)

train_data_base.sample(5)

,date,day_of_week,season,league,league_level,home_team,away_team,home_scored,away_scored,day,home_squad_size,home_average_age,home_amount_of_foreigners,home_e_market_value,home_total_market_value,away_squad_size,away_average_age,away_amount_of_foreigners,away_e_market_value,away_total_market_value,target,log_away_e_market_value,log_away_total_market_value,log_home_e_market_value,log_home_total_market_value
666,1549238400,Monday,2018-2019,premier-league,1,West Ham United,Liverpool FC,1,1,2019-02-04,37,26.6,23,9.700000e+08,3.590000e+10,39,24.7,25,3.006000e+09,1.170000e+11,1,21.823876,25.485440,20.692807,24.304003
3452,1572048000,Saturday,2019-2020,championship,2,Hull City,Derby County,2,0,2019-10-26,40,23.9,19,1.250000e+08,4.988000e+09,35,25.5,12,1.470000e+08,5.148000e+09,3,18.805943,22.361874,18.643824,22.330301
1938,1475280000,Saturday,2016-2017,championship,2,Ipswich Town,Huddersfield Town,0,1,2016-10-01,42,25.0,18,6.180000e+05,2.595000e+09,32,25.0,17,1.230000e+08,3.935000e+09,0,18.627695,22.093177,13.334244,21.676852
3949,1603843200,Wednesday,2020-2021,championship,2,Luton Town,Nottingham Forest,1,1,2020-10-28,39,25.3,14,6.400000e+05,2.495000e+09,38,27.4,23,1.460000e+08,5.545000e+09,1,18.799117,22.436162,13.369223,21.637555
634,1556236800,Friday,2018-2019,premier-league,1,Liverpool FC,Huddersfield Town,5,0,2019-04-26,39,24.7,25,3.006000e+09,1.170000e+11,33,25.3,21,4.170000e+08,1.374500e+10,3,19.848597,23.343941,21.823876,25.485440


# ПЕРЕДЕЛАТЬ! Считаем сезонные total :
- забитые
- пропущенные
- количество очков

In [14]:
query = '((home_team == @team) | (away_team == @team)) & (season == @season) & (league == @league)'

season_data = pd.DataFrame()

total_features = train_data_base.copy()

for league in total_features.league.unique():

    for season in total_features.season.unique():
        
        season_league_data = total_features.query('(season == @season) & (league == @league)')
        
        for team in season_league_data.home_team.unique():   
            
            season_data = season_league_data.query('((home_team == @team) | (away_team == @team))')
            
            total_points = 0
            total_scored = 0
            total_missed = 0


            for idx in season_data.index:

                if season_data.loc[idx, 'home_team'] == team:

                    total_points += season_data.loc[idx, 'target']
                    total_scored += season_data.loc[idx, 'home_scored']
                    total_missed += season_data.loc[idx, 'away_scored']

                else:

                    home = season_data.loc[idx, 'home_scored']
                    away = season_data.loc[idx, 'away_scored']

                    away_match_score = 3 if home < away else 1 if home == away else 0

                    total_points += away_match_score
                    total_scored += season_data.loc[idx, 'away_scored']
                    total_missed += season_data.loc[idx, 'home_scored']

            condition_home = ((total_features.home_team == team) & (total_features.season == season))
            condition_away = ((total_features.away_team == team) & (total_features.season == season))
                    
            total_features.loc[condition_home,'total_points_home'] = total_points
            total_features.loc[condition_away,'total_points_away'] = total_points
            
            total_features.loc[condition_home,'total_scored_home'] = total_scored
            total_features.loc[condition_away,'total_scored_away'] = total_scored

            total_features.loc[condition_home,'total_missed_home'] = total_missed
            total_features.loc[condition_away,'total_missed_away'] = total_missed

            total_features.loc[condition_home,'total_diff_home'] = total_scored - total_missed
            total_features.loc[condition_away,'total_diff_away'] = total_scored - total_missed            



            print(f"TEAM: {team} LEAGUE: {league} SEASON: {season}\ntotal points:{total_points}\ttotal scored:{total_scored}\ttotal missed:{total_missed}\ttotal_diff: {total_scored - total_missed}")
            print('_________')
        
total_features.tail()        

TEAM: Preston North End LEAGUE: championship SEASON: 2016-2017
total points:52	total scored:49	total missed:44	total_diff: 5
_________
TEAM: Sheffield Wednesday LEAGUE: championship SEASON: 2016-2017
total points:59	total scored:45	total missed:33	total_diff: 12
_________
TEAM: Leeds United LEAGUE: championship SEASON: 2016-2017
total points:61	total scored:48	total missed:32	total_diff: 16
_________
TEAM: Reading FC LEAGUE: championship SEASON: 2016-2017
total points:66	total scored:53	total missed:46	total_diff: 7
_________
TEAM: Wolverhampton Wanderers LEAGUE: championship SEASON: 2016-2017
total points:40	total scored:39	total missed:43	total_diff: -4
_________
TEAM: Norwich City LEAGUE: championship SEASON: 2016-2017
total points:50	total scored:64	total missed:51	total_diff: 13
_________
TEAM: Rotherham United LEAGUE: championship SEASON: 2016-2017
total points:13	total scored:27	total missed:71	total_diff: -44
_________
TEAM: Aston Villa LEAGUE: championship SEASON: 2016-2017
to

TEAM: Hull City LEAGUE: championship SEASON: 2019-2020
total points:38	total scored:47	total missed:76	total_diff: -29
_________
TEAM: Fulham FC LEAGUE: championship SEASON: 2019-2020
total points:71	total scored:61	total missed:46	total_diff: 15
_________
TEAM: Queens Park Rangers LEAGUE: championship SEASON: 2019-2020
total points:55	total scored:65	total missed:72	total_diff: -7
_________
TEAM: Sheffield Wednesday LEAGUE: championship SEASON: 2019-2020
total points:53	total scored:54	total missed:57	total_diff: -3
_________
TEAM: Preston North End LEAGUE: championship SEASON: 2019-2020
total points:59	total scored:52	total missed:49	total_diff: 3
_________
TEAM: Leeds United LEAGUE: championship SEASON: 2019-2020
total points:83	total scored:65	total missed:33	total_diff: 32
_________
TEAM: Cardiff City LEAGUE: championship SEASON: 2019-2020
total points:68	total scored:64	total missed:55	total_diff: 9
_________
TEAM: Derby County LEAGUE: championship SEASON: 2019-2020
total points:

TEAM: Stoke City LEAGUE: premier-league SEASON: 2017-2018
total points:28	total scored:26	total missed:40	total_diff: -14
_________
TEAM: Liverpool FC LEAGUE: premier-league SEASON: 2017-2018
total points:56	total scored:66	total missed:26	total_diff: 40
_________
TEAM: Huddersfield Town LEAGUE: premier-league SEASON: 2017-2018
total points:30	total scored:25	total missed:43	total_diff: -18
_________
TEAM: Manchester City LEAGUE: premier-league SEASON: 2017-2018
total points:76	total scored:74	total missed:21	total_diff: 53
_________
TEAM: Tottenham Hotspur LEAGUE: premier-league SEASON: 2017-2018
total points:52	total scored:51	total missed:29	total_diff: 22
_________
TEAM: West Ham United LEAGUE: premier-league SEASON: 2017-2018
total points:27	total scored:34	total missed:56	total_diff: -22
_________
TEAM: Manchester United LEAGUE: premier-league SEASON: 2018-2019
total points:60	total scored:59	total missed:47	total_diff: 12
_________
TEAM: Watford FC LEAGUE: premier-league SEASON:

,date,day_of_week,season,league,league_level,home_team,away_team,home_scored,away_scored,day,home_squad_size,home_average_age,home_amount_of_foreigners,home_e_market_value,home_total_market_value,away_squad_size,away_average_age,away_amount_of_foreigners,away_e_market_value,away_total_market_value,target,log_away_e_market_value,log_away_total_market_value,log_home_e_market_value,log_home_total_market_value,total_points_home,total_points_away,total_scored_home,total_scored_away,total_missed_home,total_missed_away,total_diff_home,total_diff_away
1691,1649548800,Sunday,2021-2022,premier-league,1,Manchester City,Liverpool FC,2,2,2022-04-10,22,27.5,15,4.360000e+09,9.593000e+10,27,27.1,19,3.335000e+09,9.005000e+10,1,21.927739,25.223631,22.195738,25.286885,57.0,58.0,57.0,62.0,17.0,16.0,40.0,46.0
1702,1649548800,Sunday,2021-2022,premier-league,1,Brentford FC,West Ham United,2,0,2022-04-10,25,25.8,22,9.240000e+08,2.309000e+10,24,28.8,16,1.453000e+09,3.487500e+10,3,21.096896,24.275036,20.644223,23.862665,24.0,40.0,29.0,35.0,42.0,29.0,-13.0,6.0
1515,1649548800,Sunday,2021-2022,premier-league,1,Leicester City,Crystal Palace,2,1,2022-04-10,27,27.0,18,1.907000e+09,5.148000e+10,26,27.5,14,1.000000e+09,2.599500e+10,3,20.723266,23.981170,21.368797,24.664459,29.0,30.0,32.0,35.0,34.0,29.0,-2.0,6.0
4369,1649548800,Sunday,2021-2022,championship,2,Fulham FC,Coventry City,1,3,2022-04-10,26,26.9,20,5.650000e+08,1.470000e+10,25,25.4,10,7.960000e+05,1.990000e+09,0,13.587354,21.411400,20.152336,23.411113,51.0,39.0,62.0,38.0,24.0,35.0,38.0,3.0
4444,1649635200,Monday,2021-2022,championship,2,Huddersfield Town,Luton Town,2,0,2022-04-11,28,26.0,10,1.340000e+08,3.765000e+09,26,27.1,11,8.080000e+05,2.100000e+09,3,13.602317,21.465203,18.713350,22.049014,47.0,49.0,32.0,40.0,28.0,27.0,4.0,13.0


In [32]:
home_team_alltime = train_data_base.groupby(['home_team'], as_index=False) \
                        .agg(home_mean_scored_season = ('home_scored', 'mean'),
                            home_mean_missed_season = ('away_scored', 'mean')                                 
                            )

away_team_alltime = train_data_base.groupby(['away_team'], as_index=False) \
                        .agg(away_mean_scored_alltime = ('home_scored', 'mean'),
                            away_mean_missed_alltime = ('away_scored', 'mean')                                 
                            )

home_team_alltime.head()

,home_team,home_mean_scored_season,home_mean_missed_season
0,AFC Bournemouth,1.641304,1.293478
1,Arsenal FC,1.988889,1.044444
2,Aston Villa,1.637255,1.294118
3,Barnsley FC,1.173913,1.326087
4,Birmingham City,1.142857,1.375000


In [27]:
train_data_base = train_data_base.merge(home_team_alltime, how='left', on=['home_team', 'season'])
train_data_base = train_data_base.merge(away_team_alltime, how='left', on=['away_team', 'season'])

train_data_base.head()

,date,day_of_week,season,league,league_level,home_team,away_team,home_scored,away_scored,day,home_squad_size,home_average_age,home_amount_of_foreigners,home_e_market_value,home_total_market_value,away_squad_size,away_average_age,away_amount_of_foreigners,away_e_market_value,away_total_market_value,log_away_e_market_value,log_away_total_market_value,log_home_e_market_value,log_home_total_market_value,home_scored_by_season,home_missed_by_season,home_mean_scored_season,home_mean_missed_season,away_scored_by_season,away_missed_by_season,away_mean_scored_season,away_mean_missed_season
0,1474588800,Friday,2016-2017,championship,2,Preston North End,Wigan Athletic,1,0,2016-09-23,33,26.2,19,580000.0,1.915000e+09,57,24.5,20,530000.0,3.023000e+09,13.180632,21.829516,13.270783,21.372983,29,20,1.705882,1.176471,14,16,0.823529,0.941176
1,1474675200,Saturday,2016-2017,championship,2,Sheffield Wednesday,Nottingham Forest,2,1,2016-09-24,42,26.3,24,160000000.0,6.713000e+09,45,25.3,30,141000000.0,6.340000e+09,18.764270,22.570145,18.890684,22.627312,27,16,1.588235,0.941176,15,32,0.882353,1.882353
2,1474675200,Saturday,2016-2017,championship,2,Leeds United,Ipswich Town,1,0,2016-09-24,37,24.7,22,136000000.0,5.045000e+09,42,25.0,18,618000.0,2.595000e+09,13.334244,21.676852,18.728165,22.341663,25,11,1.388889,0.611111,13,25,0.812500,1.562500
3,1474675200,Saturday,2016-2017,championship,2,Reading FC,Huddersfield Town,1,0,2016-09-24,42,24.6,28,104000000.0,4.350000e+09,32,25.0,17,123000000.0,3.935000e+09,18.627695,22.093177,18.459901,22.193442,27,12,1.500000,0.666667,18,27,1.000000,1.500000
4,1474675200,Saturday,2016-2017,championship,2,Wolverhampton Wanderers,Brentford FC,3,1,2016-09-24,44,24.0,24,136000000.0,5.963000e+09,38,23.5,27,888000.0,3.375000e+09,13.696727,21.939661,18.728165,22.508840,18,22,1.125000,1.375000,24,31,1.500000,1.937500


# ПЕРЕДЕЛАТЬ! Личные встречи за все время забитые/пропущенные

In [ ]:
# personal_battles = train_with_names.groupby(['home_team', 'away_team'], as_index=False) \
#                         .agg(total_home_scored = ('home_scored', 'sum'),
#                             total_home_missed = ('away_scored', 'sum'),
                            
#                             )

# personal_battles

# Подсчет текущих очков и вин/луз стриков по сезонам

In [48]:
query = '((home_team == @team) | (away_team == @team)) & (league == @season)'

def calculate_win_streak(actual_win_streak: int, match_result: int) -> int:
    
    new_win_streak = actual_win_streak
    
    if match_result == 3:
        
        new_win_streak += 1
    
    else:
        
        new_win_streak = 0
        
    return new_win_streak

def calculate_lose_streak(actual_lose_streak: int, match_result: int) -> int:
    
    new_lose_streak = actual_lose_streak
    
    if match_result == 0:
        
        new_lose_streak += 1
    
    else:
        
        new_lose_streak = 0
        
    return new_lose_streak


train_with_current_points = train_data_base.copy()

for season in train_with_current_points.season.unique():
    
    for team in train_with_current_points.home_team.unique():    

        current_points = 0
        current_win_streak = 0
        current_lose_streak = 0

        team_season_data = train_with_current_points.query(query)

        for idx in team_season_data.index:
            
#             if season == 'premier-league-2021-2022' and team == 'Arsenal':

#                 print(f"Match of {team_season_data.loc[idx, 'date']}\n"
#                          f"Home team: {team_season_data.loc[idx, 'home_team']}, "
#                          f"away team: {team_season_data.loc[idx, 'away_team']}"
#                          )

            if team_season_data.loc[idx, 'home_team'] == team:

                train_with_current_points.loc[idx, 'home_current_points'] = current_points

                current_points += team_season_data.loc[idx, 'target']

#                 if season == 'premier-league-2021-2022' and team == 'Arsenal':
    
#                     print(f"result is {team_season_data.loc[idx, 'target']} points for home team {team_season_data.loc[idx, 'home_team']}\n"
#                          f"Match score {team_season_data.loc[idx, 'home_scored']}:{team_season_data.loc[idx, 'away_scored']}")

                train_with_current_points.loc[idx, 'home_current_lose_streak'] = current_lose_streak

                train_with_current_points.loc[idx, 'home_current_win_streak'] = current_win_streak

                current_lose_streak = calculate_lose_streak(current_lose_streak, team_season_data.loc[idx, 'target'])

                current_win_streak = calculate_win_streak(current_win_streak, team_season_data.loc[idx, 'target'])

            else:

                train_with_current_points.loc[idx, 'away_current_points'] = current_points

                home = team_season_data.loc[idx, 'home_scored']
                away = team_season_data.loc[idx, 'away_scored']

                away_match_score = 3 if home < away else 1 if home == away else 0

                current_points += away_match_score
                
#                 if season == 'premier-league-2021-2022' and team == 'Arsenal':

#                     print(f"result is {away_match_score} points for away team {team_season_data.loc[idx, 'away_team']}\n"
#                          f"Match score {home}:{away}")

                train_with_current_points.loc[idx, 'away_current_lose_streak'] = current_lose_streak

                train_with_current_points.loc[idx, 'away_current_win_streak'] = current_win_streak

                current_lose_streak = calculate_lose_streak(current_lose_streak, away_match_score)

                current_win_streak = calculate_win_streak(current_win_streak, away_match_score)
            
#             if season == 'premier-league-2021-2022' and team == 'Arsenal':

#                 print('______________________________________________________________ \n')

train_with_current_points.home_current_points = train_with_current_points.home_current_points.astype(int)
train_with_current_points.away_current_points = train_with_current_points.away_current_points.astype(int)
train_with_current_points.away_current_win_streak = train_with_current_points.away_current_win_streak.astype(int)
train_with_current_points.away_current_lose_streak = train_with_current_points.away_current_lose_streak.astype(int)
train_with_current_points.home_current_win_streak = train_with_current_points.home_current_win_streak.astype(int)
train_with_current_points.home_current_lose_streak = train_with_current_points.home_current_lose_streak.astype(int)

AttributeError: 'DataFrame' object has no attribute 'home_current_points'

In [49]:
train_with_current_points[(train_with_current_points.home_team == 'Manchester City')|(train_with_current_points.away_team == 'Manchester City')].tail(15)

,date,day_of_week,season,league,league_level,home_team,away_team,home_scored,away_scored,day,home_squad_size,home_average_age,home_amount_of_foreigners,home_e_market_value,home_total_market_value,away_squad_size,away_average_age,away_amount_of_foreigners,away_e_market_value,away_total_market_value,target,log_away_e_market_value,log_away_total_market_value,log_home_e_market_value,log_home_total_market_value
1635,1639180800,Saturday,2021-2022,premier-league,1,Manchester City,Wolverhampton Wanderers,1,0,2021-12-11,22,27.5,15,4.360000e+09,9.593000e+10,25,25.9,21,1.408000e+09,3.520000e+10,3,21.065436,24.284312,22.195738,25.286885
1535,1639440000,Tuesday,2021-2022,premier-league,1,Manchester City,Leeds United,7,0,2021-12-14,22,27.5,15,4.360000e+09,9.593000e+10,24,24.9,15,1.200000e+09,2.880000e+10,3,20.905587,24.083641,22.195738,25.286885
1680,1639872000,Sunday,2021-2022,premier-league,1,Newcastle United,Manchester City,0,4,2021-12-19,28,28.4,15,1.039000e+09,2.909000e+10,22,27.5,15,4.360000e+09,9.593000e+10,0,22.195738,25.286885,20.761525,24.093660
1543,1640476800,Sunday,2021-2022,premier-league,1,Manchester City,Leicester City,6,3,2021-12-26,22,27.5,15,4.360000e+09,9.593000e+10,27,27.0,18,1.907000e+09,5.148000e+10,3,21.368797,24.664459,22.195738,25.286885
1679,1640736000,Wednesday,2021-2022,premier-league,1,Brentford FC,Manchester City,0,1,2021-12-29,25,25.8,22,9.240000e+08,2.309000e+10,22,27.5,15,4.360000e+09,9.593000e+10,0,22.195738,25.286885,20.644223,23.862665
1545,1640995200,Saturday,2021-2022,premier-league,1,Arsenal FC,Manchester City,1,2,2022-01-01,21,24.6,14,2.481000e+09,5.210000e+10,22,27.5,15,4.360000e+09,9.593000e+10,0,22.195738,25.286885,21.631928,24.676431
1672,1642809600,Saturday,2021-2022,premier-league,1,Southampton FC,Manchester City,1,1,2022-01-22,26,26.6,15,1.022000e+09,2.657500e+10,22,27.5,15,4.360000e+09,9.593000e+10,1,22.195738,25.286885,20.745027,24.003237
1575,1644364800,Wednesday,2021-2022,premier-league,1,Manchester City,Brentford FC,2,0,2022-02-09,22,27.5,15,4.360000e+09,9.593000e+10,25,25.8,22,9.240000e+08,2.309000e+10,3,20.644223,23.862665,22.195738,25.286885
1717,1644624000,Saturday,2021-2022,premier-league,1,Norwich City,Manchester City,0,4,2022-02-12,26,25.7,19,5.640000e+08,1.465500e+10,22,27.5,15,4.360000e+09,9.593000e+10,0,22.195738,25.286885,20.150565,23.408047
1726,1645228800,Saturday,2021-2022,premier-league,1,Manchester City,Tottenham Hotspur,2,3,2022-02-19,22,27.5,15,4.360000e+09,9.593000e+10,22,25.6,15,2.669000e+09,5.872500e+10,0,21.704970,24.796131,22.195738,25.286885


In [ ]:
train_with_names = train_with_names.merge(train_with_current_points, how='left')
train_with_names.sample(5)

# <a id='model'></a> Model  
[go to transformer](#transformer)

[go to top](#top)

In [ ]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score

In [ ]:
categorical_features.append('home_team')
categorical_features.append('away_team')
categorical_features

In [ ]:
cat_features = list(categorical_features)

model_data = {
    'train':train,
    'val':val,
    'test':test,
    'target':'target',
    'cat_features':cat_features
}

In [ ]:
X_test = test.drop(columns=['target'])
y_test = test.target

In [ ]:
model_params = {'n_estimators':1000,
                'learning_rate':0.01,
                'loss_function':'MultiClass',
                'task_type':'GPU',
                'verbose':250
                }

model = BoostingModel(params=model_params, data=model_data)

model.fit()
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)

# Оценка качества

In [ ]:
print(f'Validation ACCURACY SCORE: {accuracy_score(y_test, preds_class)}')

In [ ]:
print(f'Validation ROC AUC SCORE: {roc_auc_score(y_test, preds_proba, multi_class="ovr")}')

In [ ]:
train_preds_class = model.model.predict(train.drop(columns=['target']))

In [ ]:
train_preds_df = pd.DataFrame()
train_preds_df['pred_class'] = train_preds_class.ravel()

In [ ]:
print('True target rate: ')
print(f"Class_0 rate {train.target.value_counts()[0] / train.shape[0]}")
print(f"Class_1 rate {train.target.value_counts()[1] / train.shape[0]}")
print(f"Class_3 rate {train.target.value_counts()[3] / train.shape[0]}")

In [ ]:
print('Predict target rate:')
print(f"Class_0 rate {train_preds_df.pred_class.value_counts()[0] / train_preds_df.shape[0]}")
print(f"Class_1 rate {train_preds_df.pred_class.value_counts()[1] / train_preds_df.shape[0]}")
print(f"Class_3 rate {train_preds_df.pred_class.value_counts()[3] / train_preds_df.shape[0]}")

In [ ]:
results = pd.DataFrame()

results['true'] = y_test
results['pred_class'] = preds_class
results['score_0'] = preds_proba[:,0]
results['score_1'] = preds_proba[:,1]
results['score_3'] = preds_proba[:,2]

results

# Влияние фичей на качество модели

In [ ]:
feature_importance = model.get_feature_importances()
feature_importance_df = pd.DataFrame()

feature_importance_df['feature'] = X_test.columns
feature_importance_df['importance'] = feature_importance

good_features = feature_importance_df[np.abs(feature_importance_df.importance) > 0].sort_values(by='importance',ascending=False).head(15)
bad_features = feature_importance_df.sort_values(by='importance',ascending=True).head(15)

print(f'Number of features {X_test.shape[1]}')

In [ ]:
feature_importance_df.importance.sum()

In [ ]:
good_features

In [ ]:
bad_features

In [ ]:
test_predictions = pd.DataFrame()
test_predictions['home_team'] = X_test.home_team.map(decode_teams)
test_predictions['away_team'] = X_test.away_team.map(decode_teams)
test_predictions['home_win_proba'] = preds_proba[:,0]
test_predictions['draw_proba'] = preds_proba[:,1]
test_predictions['away_win_proba'] = preds_proba[:,2]
test_predictions['home_win_rate'] = X_test.home_win_rate
test_predictions['draw_rate'] = X_test.draw_rate
test_predictions['away_win_rate'] = X_test.away_win_rate
test_predictions['result'] = y_test
test_predictions['predict'] = preds_class

In [ ]:
test_predictions

# Считаем ROI - "коэффициент выгодности ставки"

In [ ]:
class ROIChecker():
    def __init__(self, predictions):
        self.predictions = predictions
    
    def get_roi(self):
        
        self.predictions['home_win_ROI'] = self.predictions.home_win_rate * self.predictions.home_win_proba - 1
        self.predictions['away_win_ROI'] = self.predictions.away_win_rate * self.predictions.away_win_proba - 1
        self.predictions['draw_ROI'] = self.predictions.draw_rate * self.predictions.draw_proba - 1
        
        return self.predictions

In [ ]:
checker = ROIChecker(test_predictions)
roi_info = checker.get_roi()

In [ ]:
roi_info.head()

In [ ]:
def explain_roi_info(roi_info):
    roi_cols = ['home_win_ROI', 'away_win_ROI', 'draw_ROI']
    best_roi_df = pd.DataFrame()
    for index, row in roi_info.iterrows():
        print(f"Match #{index+1}: {row.home_team} vs {row.away_team}")
        
        max_roi = np.max(roi_info.loc[index, roi_cols])
        current_choice = 'home_win_ROI'
        
        for col in roi_cols:
            if row[col] == max_roi:
                current_choice = col
        current_choice = ' '.join(current_choice.split('_')[:1])
        
        if max_roi > 0:
            print(f"Maximal ROI = {max_roi*100}% on {current_choice}")
        
        else:
            print("Нет положительного ROI")
            
        print('_______________________________ \n')
        
        best_roi_df.loc[index, 'home_team'] = row.home_team
        best_roi_df.loc[index, 'away_team'] = row.away_team
        best_roi_df.loc[index, 'best_ROI'] = max_roi
        best_roi_df.loc[index, 'choice'] = current_choice
        best_roi_df.loc[index, 'home_win_rate'] = row.home_win_rate
        best_roi_df.loc[index, 'draw_rate'] = row.draw_rate
        best_roi_df.loc[index, 'away_win_rate'] = row.away_win_rate
        best_roi_df.loc[index, 'result'] = row.result
        best_roi_df.loc[index, 'predict'] = row.predict

        
    return best_roi_df

In [ ]:
best_roi_df = explain_roi_info(roi_info)

In [ ]:
best_roi_df

In [ ]:
def money_score(best_roi_df, bet=100):
    results = {'home':0, 'draw':1, 'away':2}
    cols = {'home':'home_win_rate', 'draw':'draw_rate', 'away':'away_win_rate'}
    profit = 0
    skipped_bets = 0
    accepted_bets = 0
    
    for index, row in best_roi_df.iterrows():
        if row[cols[row.choice]] > 100:
                skipped_bets += 1
        else:
            accepted_bets += 1

            if results[row.choice] == row.result:

                current_profit = bet * (row[cols[row.choice]] - 1)
                profit += current_profit
                print(f'Match {row.home_team} vs {row.away_team}')
                print(row.choice, row[cols[row.choice]], current_profit)
                print(f'Match score ')
                print('_____________________________________________________\n')
            else:
                profit -= bet
    return profit, skipped_bets, accepted_bets
    

In [ ]:
score = money_score(best_roi_df, 100)

In [ ]:
print(f'Skipped bets: {score[1]} \nAccepted bets: {score[2]} \nProfit: {score[0]}$')

# [НАВЕРХ](#top)